In [247]:
import math
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from pandas import json_normalize
import yaml
from yaml import CSafeLoader as Loader

sns.set_context("paper", font_scale=3)

LOG_DIR = os.path.expanduser("~/jean-zay/emergent_communication/lightning_logs/")
# LOG_DIR = os.path.expanduser("~/PhD/emergent_communication/emergent_communication/lightning_logs_cluster/")

REFERENCE_METRIC = "val_acc"

In [ ]:
import torch

# Remove superfluous NaN cells
def compress(values):
    for val in values:
        if val is not None and not np.isnan(val):
            return val
        
results_df = []
for run_dir in tqdm(os.listdir(LOG_DIR)):
    run_path = os.path.join(LOG_DIR, run_dir, "checkpoints/")

    results = glob.glob(run_path+f"*{REFERENCE_METRIC}=*.pickle")
    if len(results) < 1:
        run_path = os.path.join(LOG_DIR, run_dir)
        results = glob.glob(run_path+f"/*.pickle")
        if len(results) >= 1:
            print(f"considering pre-final results for run {run_dir[8:]}")
            if run_dir[8:] == "31521":
                print("gre")

    for result in results:
        try:
            data = pd.read_pickle(result)
        except Exception as e:
            print(result)
            print(e)
            continue
            
        if isinstance(data, list):
            df = pd.DataFrame.from_records(data)
            df = df.groupby(["dir_name", "epoch"]).aggregate(compress)
        else:
            if "receiver_aux_loss" in data.keys():
                data["receiver_auxiliary_loss"] = data["receiver_aux_loss"]
                del data["receiver_aux_loss"]
            df = pd.DataFrame.from_records([{k: v.item() if torch.is_tensor(v) else v for k, v in data.items()}])
        
        df["dir_name"] = run_dir
        if "epoch" in result:
            df["epoch"] = int(result.split("epoch=")[1].split("-")[0])
        results_df.append(df)


results_df = pd.concat(results_df, ignore_index=True)

results_df.reset_index(inplace=True)
results_df

  0%|          | 0/258 [00:00<?, ?it/s]

considering pre-final results for run 42218
considering pre-final results for run 41781


  1%|          | 3/258 [00:00<00:42,  5.97it/s]

considering pre-final results for run 40703


  2%|▏         | 5/258 [00:00<00:41,  6.13it/s]

...

 12%|█▏        | 31/258 [00:06<00:51,  4.41it/s]

considering pre-final results for run 37109


 12%|█▏        | 32/258 [00:07<01:40,  2.25it/s]

considering pre-final results for run 31549


 13%|█▎        | 33/258 [00:08<02:08,  1.75it/s]

In [ ]:
"version_31521" in results_df.dir_name.values

In [ ]:
hp = []
for run_dir in tqdm(os.listdir(LOG_DIR)):
    file_path = os.path.join(LOG_DIR, run_dir, "hparams.yaml")
    file = yaml.load(open(file_path), Loader=Loader) #safe_load(, Loader=Loader)
    df = json_normalize(file)
    df["dir_name"] = run_dir
    hp.append(df)

hp = pd.concat(hp, ignore_index=True)
hp

In [ ]:
def fix_duplicate_value(val, allow_offset=None):
    if isinstance(val, list):
        for el in val:
            if allow_offset is None:
                assert (el == val[0]) or (el == "None") or (val[0] == "None")
            else:
                assert (np.abs(el - val[0]) < allow_offset) or (el == "None") or (val[0] == "None")
        return val[0]
    else:
        return val

hp = hp.applymap(fix_duplicate_value)

In [ ]:
indices_best_steps = results_df.groupby("dir_name")[REFERENCE_METRIC].idxmax()

df = results_df.loc[list(indices_best_steps)].copy()
df

In [ ]:
df.set_index("dir_name", inplace=True, drop=False)
if not hp.index.name == "dir_name":
    hp.set_index("dir_name", inplace=True, verify_integrity=True)
df = df.join(hp, how="left")

In [ ]:
assert (df.sender_entropy_coeff == df.receiver_entropy_coeff).all()
assert (df.num_senders == df.num_receivers).all()
assert (df.sender_layer_norm == df.receiver_layer_norm).all()

df["entropy_coeff"] = df["sender_entropy_coeff"]
df["num_agents"] = df["num_senders"]
df["layer_norm"] = df["sender_layer_norm"]

df["attr_val"] = df["num_attributes"].map(int).map(str) + "_" + df["num_values"].map(int).map(str)

data = df

In [ ]:
def calc_capacity(row):
    return math.pow(row.num_values, row.num_attributes)

data["capacity"] = data.apply(calc_capacity, axis=1)
data.sort_values("capacity", inplace=True)

In [ ]:
data["test_acc"] = data["val_acc"]
data["test_acc_no_noise"] = data["val_acc_no_noise"]

In [ ]:
data["condition"] = data["noise"].map(lambda x: f"noise_{x}" if x > 0 else "baseline") + data["feedback"].map(lambda x: "_feedback" if x else "") + data["receiver_aux_loss"].map(lambda x: "_aux_loss" if x else "")

target_data = data.copy()

target_data = target_data[target_data.receiver_aux_loss == False]

NUM_AGENTS = 1
target_data = target_data[target_data.num_agents == NUM_AGENTS]

MAX_LEN = 10
target_data = target_data[target_data.max_len == MAX_LEN]

NOISE = 0.5
target_data = target_data[target_data.noise == NOISE]

VOCAB_SIZE = 2
target_data = target_data[target_data.vocab_size == VOCAB_SIZE]

VOCAB_SIZE_FEEDBACK = 2
target_data = target_data[(target_data.vocab_size_feedback == VOCAB_SIZE_FEEDBACK) | (target_data.feedback == False)]

LAYER_NORM = 1
target_data = target_data[target_data.layer_norm == LAYER_NORM]

DISCRIMINATION_NUM_OBJECTS = 2
target_data = target_data[target_data.discrimination_num_objects == DISCRIMINATION_NUM_OBJECTS]

# hue_order = None
# hue_order = ["baseline", "noise_0.1"]
hue_order = ["noise_0.5", "noise_0.5_feedback", "noise_0.5_feedback_aux_loss"]

# order = ["2_10", "3_5", "2_16", "4_4", "4_5", "3_10", "2_100"]
# order = ["2_100", "4_100", "10_100"]
order = ["2_10", "4_4", "3_10", "2_100", "2_1000", "10_1000"]

target_hparam = "attr_val"

num_runs_data = target_data.groupby(["attr_val", "condition"]).size().reset_index()


metrics = ["test_acc", "test_acc_no_noise", "topsim"]
# metrics = ["test_acc", "topsim", "topsim_sender_receiver", "posdis", "bosdis", "test_acc_no_noise", "train_acc_no_noise"]
_, axes = plt.subplots(1, len(metrics), figsize=(35, 10))

for i, metric in enumerate(metrics):
    axis = axes[i]
    # g = sns.barplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", dodge=True)
    g = sns.pointplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", errorbar="se", linestyles="", dodge=True)

    if not i == 0:
        g.legend().remove()
    else:
        sns.move_legend(axis, "lower left")
    axis.set_title(metric)
    axis.set_ylabel("")
    # handles, labels = ax.get_legend_handles_labels()
    # num_conditions = int(len(handles)/2) if not hue_order else len(hue_order)

plt.tight_layout()
plt.savefig("plots/effect_of_attr_val.pdf", dpi=300)


In [ ]:
target_data = data.copy()

target_data = target_data[target_data.receiver_aux_loss == False]

NUM_AGENTS = 1
target_data = target_data[target_data.num_agents == NUM_AGENTS]

MAX_LEN = 10
target_data = target_data[target_data.max_len == MAX_LEN]

VOCAB_SIZE = 2
target_data = target_data[target_data.vocab_size == VOCAB_SIZE]

VOCAB_SIZE_FEEDBACK = 2
target_data = target_data[(target_data.vocab_size_feedback == VOCAB_SIZE_FEEDBACK) | (target_data.feedback == False)]

LAYER_NORM = 1
target_data = target_data[target_data.layer_norm == LAYER_NORM]

DISCRIMINATION_NUM_OBJECTS = 2
target_data = target_data[target_data.discrimination_num_objects == DISCRIMINATION_NUM_OBJECTS]

ATTR_VAL = "4_4"
# ATTR_VAL = "3_10"

target_data = target_data[(target_data["attr_val"] == ATTR_VAL)]

# print(target_data.dir_name.unique())
target_hparam = "noise"
order = [0, 0.1, 0.3, 0.5, 0.7, 0.9]


num_runs_data = target_data.groupby(["attr_val", "condition"]).size().reset_index()

metrics = ["test_acc", "test_acc_no_noise", "topsim", "posdis", "bosdis"]
# metrics = ["test_acc", "topsim", "topsim_sender_receiver", "posdis", "bosdis", "test_acc_no_noise", "train_acc_no_noise"]
_, axes = plt.subplots(1, len(metrics), figsize=(35, 10))

for i, metric in enumerate(metrics):
    axis = axes[i]
    # g = sns.barplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", dodge=True)
    g = sns.pointplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", errorbar="se")

    if not i == 0:
        g.legend().remove()
    else:
        sns.move_legend(axis, "lower left")
    axis.set_title(metric)
    axis.set_ylabel("")

plt.tight_layout()
plt.savefig("plots/effect_of_noise.pdf", dpi=300)

In [ ]:
target_data = data.copy()

target_data = target_data[target_data.receiver_aux_loss == False]

NUM_AGENTS = 1
target_data = target_data[target_data.num_agents == NUM_AGENTS]

NOISE = 0.7
target_data = target_data[target_data.noise == NOISE]

VOCAB_SIZE = 2
target_data = target_data[target_data.vocab_size == VOCAB_SIZE]

VOCAB_SIZE_FEEDBACK = 2
target_data = target_data[(target_data.vocab_size_feedback == VOCAB_SIZE_FEEDBACK) | (target_data.feedback == False)]

LAYER_NORM = 1
target_data = target_data[target_data.layer_norm == LAYER_NORM]

DISCRIMINATION_NUM_OBJECTS = 2
target_data = target_data[target_data.discrimination_num_objects == DISCRIMINATION_NUM_OBJECTS]

ATTR_VAL = "4_4"

target_data = target_data[(target_data["attr_val"] == ATTR_VAL)]

# target_hparam = "attr_val"
# order = ["2_10", "4_4", "2_100", "2_1000"]
target_hparam = "max_len"
order = [1, 3, 5, 10, 20, 30]


num_runs_data = target_data.groupby(["attr_val", "condition"]).size().reset_index()

metrics = ["test_acc", "test_acc_no_noise", "topsim"]
# metrics = ["test_acc", "topsim", "topsim_sender_receiver", "posdis", "bosdis", "test_acc_no_noise", "train_acc_no_noise"]
_, axes = plt.subplots(1, len(metrics), figsize=(35, 10))

for i, metric in enumerate(metrics):
    axis = axes[i]
    # g = sns.barplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", dodge=True)
    g = sns.pointplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", errorbar="se")

    if not i == 0:
        g.legend().remove()
    else:
        sns.move_legend(axis, "lower left")
    axis.set_title(metric)
    axis.set_ylabel("")

plt.tight_layout()
plt.savefig("plots/effect_of_message_length.pdf", dpi=300)

In [ ]:
target_data = data.copy()

target_data = target_data[target_data.guesswhat == True]

target_data = target_data[target_data.receiver_output_attention == False]
target_data = target_data[target_data.receiver_object_attention == False]
target_data = target_data[target_data.sender_attention == False]

target_data = target_data[(target_data.receiver_aux_loss == False)]

NUM_AGENTS = 1
target_data = target_data[target_data.num_agents == NUM_AGENTS]

MAX_LEN = 10
target_data = target_data[target_data.max_len == MAX_LEN]

# NOISE = 0.5
# target_data = target_data[target_data.noise == NOISE]

VOCAB_SIZE = 2
target_data = target_data[target_data.vocab_size == VOCAB_SIZE]

VOCAB_SIZE_FEEDBACK = 2
target_data = target_data[(target_data.vocab_size_feedback == VOCAB_SIZE_FEEDBACK) | (target_data.feedback == False)]

LAYER_NORM = 1
target_data = target_data[target_data.layer_norm == LAYER_NORM]

DISCRIMINATION_NUM_OBJECTS = 10
target_data = target_data[target_data.discrimination_num_objects == DISCRIMINATION_NUM_OBJECTS]

# target_hparam = "attr_val"
# order = ["2_10", "4_4", "2_100", "2_1000"]
target_hparam = "noise"
order = [0, 0.1, 0.3, 0.5, 0.7, 0.9]

metrics = ["test_acc", "test_acc_no_noise", "train_acc"]
# metrics = ["test_acc", "topsim", "topsim_sender_receiver", "posdis", "bosdis", "test_acc_no_noise", "train_acc_no_noise"]
_, axes = plt.subplots(1, len(metrics), figsize=(35, 10))

for i, metric in enumerate(metrics):
    axis = axes[i]
    # g = sns.barplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", dodge=True)
    g = sns.pointplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", errorbar="se")

    if not i == 0:
        g.legend().remove()
    else:
        sns.move_legend(axis, "lower left")
    axis.set_title(metric)
    axis.set_ylabel("")
    axis.set_ylim((0, 1))

plt.tight_layout()
plt.savefig("plots/guesswhat_noise.pdf", dpi=300)

In [ ]:
target_data = data.copy()

target_data = target_data[target_data.guesswhat == True]

target_data = target_data[target_data.receiver_output_attention == False]
target_data = target_data[target_data.receiver_object_attention == False]
target_data = target_data[target_data.sender_attention == False]

target_data = target_data[target_data.receiver_aux_loss == False]


NUM_AGENTS = 1
target_data = target_data[target_data.num_agents == NUM_AGENTS]

NOISE = 0.5
target_data = target_data[target_data.noise == NOISE]

VOCAB_SIZE = 2
target_data = target_data[target_data.vocab_size == VOCAB_SIZE]

VOCAB_SIZE_FEEDBACK = 2
target_data = target_data[(target_data.vocab_size_feedback == VOCAB_SIZE_FEEDBACK) | (target_data.feedback == False)]

LAYER_NORM = 1
target_data = target_data[target_data.layer_norm == LAYER_NORM]

DISCRIMINATION_NUM_OBJECTS = 10
target_data = target_data[target_data.discrimination_num_objects == DISCRIMINATION_NUM_OBJECTS]

# target_hparam = "attr_val"
# order = ["2_10", "4_4", "2_100", "2_1000"]
target_hparam = "max_len"
order = [1, 5, 10, 20, 30, 50]

metrics = ["test_acc", "test_acc_no_noise", "train_acc"]
# metrics = ["test_acc", "topsim", "topsim_sender_receiver", "posdis", "bosdis", "test_acc_no_noise", "train_acc_no_noise"]
_, axes = plt.subplots(1, len(metrics), figsize=(35, 10))

for i, metric in enumerate(metrics):
    axis = axes[i]
    # g = sns.barplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", dodge=True)
    g = sns.pointplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", errorbar="se")

    if not i == 0:
        g.legend().remove()
    else:
        sns.move_legend(axis, "lower left")
    axis.set_title(metric)
    axis.set_ylabel("")
    axis.set_ylim((0, 1))

plt.tight_layout()
plt.savefig("plots/guesswhat_max_len.pdf", dpi=300)

In [ ]:
target_data = data.copy()

target_data = target_data[target_data.guesswhat == True]

target_data = target_data[target_data.receiver_output_attention == False]
target_data = target_data[target_data.receiver_object_attention == False]
target_data = target_data[target_data.sender_attention == False]

target_data = target_data[(target_data.receiver_aux_loss == False)]

NUM_AGENTS = 1
target_data = target_data[target_data.num_agents == NUM_AGENTS]

MAX_LEN = 10
target_data = target_data[target_data.max_len == MAX_LEN]

VOCAB_SIZE = 10
target_data = target_data[target_data.vocab_size == VOCAB_SIZE]

VOCAB_SIZE_FEEDBACK = 2
target_data = target_data[(target_data.vocab_size_feedback == VOCAB_SIZE_FEEDBACK) | (target_data.feedback == False)]

LAYER_NORM = 1
target_data = target_data[target_data.layer_norm == LAYER_NORM]

DISCRIMINATION_NUM_OBJECTS = 10
target_data = target_data[target_data.discrimination_num_objects == DISCRIMINATION_NUM_OBJECTS]

# target_hparam = "attr_val"
# order = ["2_10", "4_4", "2_100", "2_1000"]
target_hparam = "noise"
order = [0, 0.5]

metrics = ["test_acc", "test_acc_no_noise", "train_acc"]
# metrics = ["test_acc", "topsim", "topsim_sender_receiver", "posdis", "bosdis", "test_acc_no_noise", "train_acc_no_noise"]
_, axes = plt.subplots(1, len(metrics), figsize=(35, 10))

for i, metric in enumerate(metrics):
    axis = axes[i]
    # g = sns.barplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", dodge=True)
    g = sns.pointplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", errorbar="se")

    if not i == 0:
        g.legend().remove()
    else:
        sns.move_legend(axis, "lower left")
    axis.set_title(metric)
    axis.set_ylabel("")
    axis.set_ylim((0, 1))

plt.tight_layout()
plt.savefig("plots/guesswhat_noise_vocab_high.pdf", dpi=300)